In [ ]:
# parametric knowledge -- data on which it was trained
# source knowledge -- additional information provided through prompts

In [ ]:
%pip install -qU langchain langchain-community
%pip install -qU datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
# Loading Dataset

In [ ]:
from datasets import load_dataset
data = load_dataset("wikipedia", "20220301.simple", split='train[:10000]')    # sample text dataset from wikipedia

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/205328 [00:00<?, ? examples/s]

In [ ]:
data[9]

{'id': '18',
 'url': 'https://simple.wikipedia.org/wiki/Andouille',
 'title': 'Andouille',
 'text': 'Andouille is a type of pork sausage. It is spicy (hot in taste) and smoked. There are different kinds, all with different combinations of pork meat, fat, intestines (tubes going to the stomach), and tripe (the wall of the stomach).\n\nOther sorts are "French andouille" and "German andouille"; they are less spicy than Cajun. Cajun has extra salt, black pepper, and garlic. Andouille makers smoke the sausages over pecan wood and sugar cane for a maximum of seven or eight hours, at about 175 degrees Fahrenheit (80 degrees Celsius).\n\nSausage'}

In [ ]:
# Creating Chunks

In [ ]:
!pip install -U tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding('p50k_base')

In [ ]:
# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap = 5,
    length_function = tiktoken_len,
    separators = ["\n\n", "\n", " ", ""]
)

In [ ]:
chunks = []
for i in range(len(data) - 9000):
    chunk = text_splitter.split_text(data[i]['text'])[:1]
    new_ch = ''.join(chunk)
    chunks.append(new_ch)

chunks

['April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.',
 'August (Aug.) is the eighth month of the year in the Gregorian calendar, coming between July and September. It has 31 days. It is named after the Roman emperor Augustus Caesar.',
 "Art is a creative activity that expresses imaginative or technical skill. It produces a product, an object. Art is a diverse range of human activities in creating visual, performing artifacts, and expressing the author's imaginative mind. The product of art is called a",
 'A or a is the first letter of the English alphabet. The small letter, a or α, is used as a lower case vowel.',
 "Air refers to the Earth's atmosphere. Air is a mixture of many gases and tiny dust particles. It is the clear gas in which living things live and breathe. It has an indefinite shape and volume. It has mass and weight, because it is",
 'Spain is divided in 17 parts called au

In [ ]:
tiktoken_len(str(chunks[8]))

50

In [ ]:
# Creating embeddings

In [ ]:
%pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = "text-embedding-ada-002"

from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")

embed = OpenAIEmbeddings(
    openai_api_key=api_key
)

In [ ]:
res = embed.embed_documents(chunks)
len(res), len(res[0])

(1000, 1536)

In [ ]:
# Setting up Database

In [ ]:
%pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import pinecone

pc = Pinecone(
    api_key="95602d09-4dc2-488e-bc0c-5b4a2323cefd"
)

In [ ]:
# Connect with the Index

index_name = "my-index"
index = pc.Index(index_name)

In [ ]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []

for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    metadata = {
        'wiki-id': str(record['id']),
        'source': record['url'],
        'title': record['title']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['text'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

  0%|          | 0/10000 [00:00<?, ?it/s]

PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 31 May 2024 07:59:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '166', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '238', 'x-pinecone-request-id': '8325398099989466545', 'x-envoy-upstream-service-time': '9', 'server': 'envoy'})
HTTP response body: {"code":8,"message":"Request failed. You've reached your write unit limit for the current month (2000000). To continue writing data, upgrade your plan.","details":[]}


In [ ]:
%pip install --upgrade langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 341315}},
 'total_vector_count': 341315}

In [ ]:
from langchain_pinecone import PineconeVectorStore

text_field = "text"
host = "Aqsa Ashfaq"
index_name="index"

# switch back to normal index for langchain
index = pinecone.Index(index_name, host=host)

vectorstore = PineconeVectorStore(
    index, embed.embed_query, text_field
)

In [ ]:
# Generative Question Answering

In [ ]:
%pip install -qU langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [ ]:
query = "Dummy"
qa_with_sources(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AttributeError: 'function' object has no attribute 'embed_query'

Facing Trouble Here -- With retrieval from the chain